<a href="https://colab.research.google.com/github/alouvre/stunting/blob/main/Stunting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

data kaggle = https://www.kaggle.com/datasets/uniksh17/dataset-posyandu-mawar

In [134]:
# !git clone https://github.com/alouvre/stunting.git

In [135]:
!ls stunting

data_posyandu  data_posyandu.zip  data_unicef  README.md  Stunting.ipynb


In [136]:
import pandas as pd
import numpy as np
from scipy import stats

# Data Mentah dari Kaggle

In [137]:
data = pd.read_excel('stunting/data_posyandu/data posyandu bulan november 2020.xlsx')
data.head()

,Unnamed: 0,data posyandu rt.01/rw.03 candimulyo,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,subject,umur,berat badan (kg),tinggi badan (cm)
1,NaN,1,3 tahun,12,80
2,NaN,2,4 tahun,18,79
3,NaN,3,1 tahun,11,76
4,NaN,4,6 bulan,7,69


# Data Processing

In [138]:
data = data.drop('Unnamed: 0', axis=1, errors='ignore')

new_column_names = ['SUBJECT', 'UMUR', 'BB', 'TB']
data.columns = new_column_names

data = data.drop(0)

# Mengubah tipe data kolom 'umur' dari float menjadi string
data['UMUR'] = data['UMUR'].astype(str)

In [139]:
data.head()

,SUBJECT,UMUR,BB,TB
1,1,3 tahun,12,80
2,2,4 tahun,18,79
3,3,1 tahun,11,76
4,4,6 bulan,7,69
5,5,5 tahun,15,102


In [140]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 1 to 50
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   SUBJECT  50 non-null     object
 1   UMUR     50 non-null     object
 2   BB       50 non-null     object
 3   TB       50 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


In [141]:
# Fungsi untuk mengonversi umur ke dalam bulan
def convert_umur_to_bulan(umur):
    umur_split = umur.split()
    if len(umur_split) == 2:
        if 'tahun' in umur_split[1].lower():
            return int(umur_split[0]) * 12
        elif 'bulan' in umur_split[1].lower():
            return int(umur_split[0])
    elif len(umur_split) == 1 and umur_split[0].lower() == 'bulan':
        return 1  # Misalkan 1 bulan untuk umur yang hanya disebutkan dalam bulan
    return None

# Mengonversi kolom 'umur' ke dalam bulan
data['UMUR'] = data['UMUR'].apply(convert_umur_to_bulan)

In [142]:
data.head()

,SUBJECT,UMUR,BB,TB
1,1,36,12,80
2,2,48,18,79
3,3,12,11,76
4,4,6,7,69
5,5,60,15,102


In [143]:
# Ubah tipe data kolom 'UMUR', 'BB', dan 'TB' menjadi numerik
data['UMUR'] = pd.to_numeric(data['UMUR'], errors='coerce')
data['BB'] = pd.to_numeric(data['BB'], errors='coerce')
data['TB'] = pd.to_numeric(data['TB'], errors='coerce')

# Pastikan tipe data telah berubah menjadi float
data['UMUR'] = data['UMUR'].astype(float)
data['BB'] = data['BB'].astype(float)
data['TB'] = data['TB'].astype(float)

# Cek tipe data
data.dtypes

SUBJECT     object
UMUR       float64
BB         float64
TB         float64
dtype: object

In [144]:
data.head()

,SUBJECT,UMUR,BB,TB
1,1,36.0,12.0,80.0
2,2,48.0,18.0,79.0
3,3,12.0,11.0,76.0
4,4,6.0,7.0,69.0
5,5,60.0,15.0,102.0


# Data Labeling

- Stunting = 1
- Bukan Stunting = 0

In [145]:
# Fungsi untuk menghitung Z-Score TB/U berdasarkan umur, berat badan, dan tinggi badan
def calculate_z_score_TBU(umur, berat_badan, tinggi_badan):
    # Menghitung Z-Score untuk umur, berat badan, dan tinggi badan
    z_score_umur = stats.zscore(umur)
    z_score_berat_badan = stats.zscore(berat_badan)
    z_score_tinggi_badan = stats.zscore(tinggi_badan)

    # Tambahkan rumus yang sesuai untuk menghitung Z-Score TB/U berdasarkan umur, berat badan, dan tinggi badan
    z_score_tb_u = z_score_umur + z_score_berat_badan + z_score_tinggi_badan
    return z_score_tb_u

data['TB/U'] = calculate_z_score_TBU(data['UMUR'], data['BB'], data['TB'])

In [146]:
# Menentukan batas kategori
batas_pendek = -3
batas_normal_bawah = -2
batas_normal_atas = 2
batas_tinggi = 2

# Menentukan kategori berdasarkan Z-Score TB/U
data['Kategori'] = pd.cut(data['TB/U'],
                               bins=[float('-inf'), batas_pendek, batas_normal_bawah, batas_normal_atas, float('inf')],
                               labels=['Sangat Pendek', 'Pendek', 'Normal', 'Tinggi'],
                               right=False)

In [147]:
# Memilih kolom yang ingin di-One-Hot Encoding
kolom_kategori = ['Kategori']  # Gantilah dengan nama kolom yang sesuai

# Melakukan One-Hot Encoding
data_encoded = pd.get_dummies(data, columns=kolom_kategori)

# Menampilkan DataFrame yang sudah di-One-Hot Encoding
data_encoded.head()

,SUBJECT,UMUR,BB,TB,TB/U,Kategori_Sangat Pendek,Kategori_Pendek,Kategori_Normal,Kategori_Tinggi
1,1,36.0,12.0,80.0,0.475087,0,0,1,0
2,2,48.0,18.0,79.0,2.693852,0,0,0,1
3,3,12.0,11.0,76.0,-1.451838,0,0,1,0
4,4,6.0,7.0,69.0,-3.342734,1,0,0,0
5,5,60.0,15.0,102.0,4.171345,0,0,0,1


In [148]:
# Menentukan batas -2 SD untuk mendefinisikan stunting
batas_stunting = -2

# Menentukan label berdasarkan Z-score TB/U
data_encoded['Label'] = data_encoded['TB/U'].apply(lambda x: '1' if x < batas_stunting else '0')

In [149]:
data_encoded.head()

,SUBJECT,UMUR,BB,TB,TB/U,Kategori_Sangat Pendek,Kategori_Pendek,Kategori_Normal,Kategori_Tinggi,Label
1,1,36.0,12.0,80.0,0.475087,0,0,1,0,0
2,2,48.0,18.0,79.0,2.693852,0,0,0,1,0
3,3,12.0,11.0,76.0,-1.451838,0,0,1,0,0
4,4,6.0,7.0,69.0,-3.342734,1,0,0,0,1
5,5,60.0,15.0,102.0,4.171345,0,0,0,1,0


# Imbalance Dataset

In [150]:
data_encoded['Label'].value_counts()

0    33
1    17
Name: Label, dtype: int64